# Deep Learning HW01
Gerald Baulig 0780827 2019/11/03

In this assignment we are asked to implement a neural network from scratch,
including Backpropagation and Storastic Gradient Decent algorithms.
The assigment includes two tasks:

1. Implement a regression network, and
2. implement a classification network.

## 1. Implement a Regression Network

Implement a regression network for the engery efficiency dataset.
I guess this dataset descripes the termal exchange of a building to its environment.
The dataset has 2 Values (t):

- Heating load, and
- Cooling load

And 8 features:

- Relative compactness,
- Surface area,
- Wall area,
- Roof area,
- Overall hight,
- Orientation,
- Glazing area, and
- Glazing area distribution.

We are required to define a feature-vector out of these features and predict the heating load of the buildings, by minimizing the "sum-of-squares" error function,

$$
\begin{equation}
E(w) = \sum_{n=1}^{N}{(t_n - y(X_n;w))^2}
\tag{1}
\end{equation}
$$

...while the evaluation should be processed by the "root-mean-square" error (basically an Euclidean distance :-),

$$
\begin{equation}
E_{RMS}(w) = \frac{\sqrt{\sum_{n=1}^{N}{(t_n - y(x_n;w))^2}}}{\sqrt{N}}
\tag{2}
\end{equation}
$$

## 2. Implement a Classifier

Implement a classification network for the Ionosphere dataset.



# A Fully-Connected Neural Network from Scratch

First, let's define what a Neural Network is.
A Neural Network is a class that can train and validate the weight of a stack of layers, w.r.t several hyper-parameters.
However, all what the Neural Network needs to know about his own network is the Input-Layer. Furthermore, the hyper-parameters are only required in the training process, but only few of them in the validation process,
such that the class defination could look like that:

In [24]:
class NeuralNetwork:
    def __init__(self, input_layer):
        self.input_layer = input_layer
        pass
    
def train(self, min_batch=1, epochs=1, lr=1):
    print("train")
    pass

NeuralNetwork.train = train

In [8]:
class FcLayer:
    def __init__(self, weights, biases, act_func):
        self.w = weights
        self.b = biased
        self.af = act_func
        pass
    
    def forward(self, x):
        

train


In [53]:
import numpy as np
def ReLU():
    ReLU.f = lambda x: np.maximum(x,0).astype(float)
    ReLU.d = lambda x: (np.array(x) > 0).astype(float)
    pass
ReLU() #init

#test
print(ReLU.f([1,-1,5,-5,0]))
print(ReLU.d([1,-1,5,-5,0]))

[1. 0. 5. 0. 0.]
[1. 0. 1. 0. 0.]


In [68]:
import numpy as np
class Sigmoid():
    f = lambda x: 1 / (1 + np.exp(-np.array(x)))
    def d(x):
        S = Sigmoid.f(x)
        return S * (1 - S)
    pass
Sigmoid() #init

#test
print(Sigmoid.f([1,-1,5,-5,0]))
print(Sigmoid.d([1,-1,5,-5,0]))

[0.73105858 0.26894142 0.99330715 0.00669285 0.5       ]
[0.19661193 0.19661193 0.00664806 0.00664806 0.25      ]
